In [ ]:
import torch.nn as nn
import torch
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
torch.manual_seed(1)
np.random.seed(1)
random.seed(1)       

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

class FeatureDataset(Dataset):
    '''
    Args: x is a 2D numpy array [x_size, x_features]
    '''
    def __init__(self, x):
        self.x = x
    
    def __len__(self):
        return self.x.shape[0]
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.x[idx])

    def getBatch(self, idxs=[]):
        if idxs == None:
            return idxs
        else:
            x_features = []
            for i in idxs:
                x_features.append(self.__getitem__(i))
            return torch.FloatTensor(x_features)

def normalizing_data(data, seed=1):  
    
    composition = data[['Ba', 'Ca', 'Sr', 'Ti', 'Zr','Sn', 'Hf']]
    descriptors = data[['W', 'EI', 'EA', 'μ']]
    
    
    min_max_scaler = MinMaxScaler()
    normalized_composition = min_max_scaler.fit_transform(composition)
    normalized_descriptors = min_max_scaler.fit_transform(descriptors)
    
    
    normalized_composition_df = pd.DataFrame(normalized_composition, columns=composition.columns)
    normalized_descriptors_df = pd.DataFrame(normalized_descriptors, columns=descriptors.columns)  
    
    
    x = pd.concat([normalized_composition_df, normalized_descriptors_df], axis=1)
    print(x)
    
    y = data[['d33(pC/N)']] 
    print(y)

    
    x = torch.FloatTensor(x.values)
    y = torch.FloatTensor(y.values)

    if torch.cuda.is_available():
        x = x.cuda()
        y = y.cuda()
    
    
    train_features, test_features, train_labels, test_labels = train_test_split(x, y, test_size=0.2, random_state=seed)
    print(y)
    return x, y, train_features, test_features, train_labels, test_labels

In [ ]:
import os
import time
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.svm import SVR
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import random
import torch

def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_random_seed(1)
starttime = datetime.datetime.now()


t = time.localtime()
model_name = 'd33_inference_SVR'
file_name = '{}.xlsx'.format(model_name)
data = pd.read_excel('data-1.xlsx')

x_all, y_all, train_features, test_features, train_labels, test_labels = normalizing_data(data, seed=1)
train_features, test_features = train_features.cpu().data.numpy(), test_features.cpu().data.numpy()
train_labels, test_labels = train_labels.cpu().data.numpy(), test_labels.cpu().data.numpy()
train_labels, test_labels = train_labels.reshape(-1), test_labels.reshape(-1) 


def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))


def train_model(C, epsilon, gamma):
    params = {
        "C": C,
        "epsilon": epsilon,
        "gamma": gamma,
        "kernel": "rbf" 
    }
    model = SVR(**params)
    
    
    scores = cross_val_score(model, train_features, train_labels, cv=5, scoring='neg_mean_absolute_percentage_error')
    cv_scores = scores
    cv_score = -scores.mean()
    print("cv_scores (each fold):", cv_scores)  
    print("cv_score (mean):", cv_score)  
    

    
    model.fit(train_features, train_labels)
    y_pred_train = model.predict(train_features)
    y_pred_test = model.predict(test_features)
    train_mape = mean_absolute_percentage_error(train_labels, y_pred_train)
    test_mape = mean_absolute_percentage_error(test_labels, y_pred_test)
    print("train_mapre:", train_mape)
    print("test_mapre:", test_mape)
    error = -mean_absolute_percentage_error(test_labels, y_pred_test)
    return error
    
    
    

bounds = {
    'C': (1, 500), 
    'epsilon': (0.1, 5), 
    'gamma': (0.01, 10) 
}

optimizer = BayesianOptimization(
    f=train_model,
    pbounds=bounds,
    random_state=1,
)


optimizer.maximize(init_points=100, n_iter=150)


result_list = []
for res in optimizer.res:
    result_list.append(pd.DataFrame({'target': [res['target']],
                                     'C': [res['params']['C']],
                                     'epsilon': [res['params']['epsilon']],
                                     'gamma': [res['params']['gamma']]}))


table = pd.concat(result_list, ignore_index=True)


best_result = pd.DataFrame({'target': [optimizer.max['target']],
                            'C': [optimizer.max['params']['C']],
                            'epsilon': [optimizer.max['params']['epsilon']],
                            'gamma': [optimizer.max['params']['gamma']]})


table = pd.concat([table, best_result], ignore_index=True)


table.to_excel(file_name)
endtime = datetime.datetime.now()
print('Running time: {}'.format(endtime - starttime))
print(table)

           Ba        Ca   Sr     Ti     Zr        Sn    Hf         W  \
0    1.000000  0.000000  0.0  1.000  0.000  0.000000  0.00  1.000000   
1    1.000000  0.000000  0.0  0.950  0.050  0.000000  0.00  0.978718   
2    1.000000  0.000000  0.0  0.950  0.000  0.071429  0.00  0.965424   
3    1.000000  0.000000  0.0  0.925  0.075  0.000000  0.00  0.968220   
4    1.000000  0.000000  0.0  0.900  0.000  0.000000  0.10  0.877303   
..        ...       ...  ...    ...    ...       ...   ...       ...   
149  0.333333  0.666667  0.0  1.000  0.000  0.000000  0.00  0.664214   
150  0.333333  0.666667  0.0  0.500  0.500  0.000000  0.00  0.495194   
151  0.333333  0.666667  0.0  0.500  0.000  0.000000  0.50  0.227934   
152  0.200000  0.800000  0.0  0.840  0.000  0.000000  0.16  0.439015   
153  0.000000  1.000000  0.0  1.000  0.000  0.000000  0.00  0.496321   

           EI        EA         μ  
0    0.885689  0.786531  0.000000  
1    0.892040  0.765946  0.016596  
2    0.868796  0.725267  0.

In [ ]:
import os
import time
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, r2_score
from sklearn.svm import SVR
import joblib
from sklearn.preprocessing import StandardScaler


def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


data = pd.read_excel('data-1.xlsx')
x_all, y_all, train_features, test_features, train_labels, test_labels = normalizing_data(data, seed=1)
train_features, test_features = train_features.cpu().data.numpy(), test_features.cpu().data.numpy()
train_labels, test_labels = train_labels.cpu().data.numpy(), test_labels.cpu().data.numpy()
train_labels, test_labels = train_labels.reshape(-1), test_labels.reshape(-1)


scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

results_df = pd.DataFrame(columns=['Iteration', 'target', 'R2_Score_test', 'R2_Score_train', 'Train Loss', 'Test Loss', 'Figure_Path_Test', 'Figure_Path_Train', 'Figure_Path_All', 'Loss_Path'])
set_random_seed(1)


results_dir = 'Results/STU_SVR_BO(100+150)_1'
figures_dir = os.path.join(results_dir, 'Figures')
ensure_directory_exists(results_dir)
ensure_directory_exists(figures_dir)


for mm in range(0, 251):
    set_random_seed(1)
    target = pd.read_excel('d33_inference_SVR.xlsx')
    tg = target.at[mm, 'target']
    C = target.at[mm, 'C']
    epsilon = target.at[mm, 'epsilon']
    gamma = target.at[mm, 'gamma']
    
    params = {
        'C': C,
        'epsilon': epsilon,
        'gamma': gamma,
        'kernel': 'rbf'
    }

    
    model = SVR(**params)
    model.fit(train_features, train_labels)
    
    model_save_path = f'Results/STU_SVR_BO(100+150)_1/{mm}-seed_1.joblib'
    joblib.dump(model, model_save_path)  
    
    
    predict_train = model.predict(train_features)
    train_mape = mean_absolute_percentage_error(train_labels, predict_train)
    predict_test = model.predict(test_features)
    test_mape = mean_absolute_percentage_error(test_labels, predict_test)
    
    plt.figure()
    plt.plot([train_mape], 'bo-', label='Train MAPE')
    plt.plot([test_mape], 'ro-', label='Test MAPE')
    plt.title('MAPE during Training')
    plt.xlabel('Iterations')
    plt.ylabel('MAPE')
    plt.text(0, test_mape, f'Target={tg:.4f}', fontdict={'size': 12, 'color': 'red'})
    plt.legend()
    plt.savefig(f'{figures_dir}/{mm}_SVR_training_history.png', format='png', dpi=300)
    plt.close()
    
    
    df_losses = pd.DataFrame({
        'Epoch': [1],
        'Train Loss': [train_mape],
        'Test Loss': [test_mape]
    })
    excel_path = os.path.join(results_dir, f'{mm}_SVR_loss_data.xlsx')
    df_losses.to_excel(excel_path, index=False)
    
    
    fig_name_test = f'{figures_dir}/{mm}_SVR_experiment_vs_pred_test.png'
    fig_name_train = f'{figures_dir}/{mm}_SVR_experiment_vs_pred_train.png'
    fig_name_all = f'{figures_dir}/{mm}_SVR_experiment_vs_pred_all.png'

    
    plt.figure()
    sns.regplot(x=predict_test, y=test_labels, color='red')
    current_r2_test = r2_score(test_labels, predict_test)
    plt.text(min(predict_test), max(test_labels), f'R²={current_r2_test:.4f}', color='red')
    plt.title('Test Prediction vs Actual')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(fig_name_test, format='png', dpi=300)
    plt.close()
    
    
    plt.figure()
    sns.regplot(x=predict_train, y=train_labels, color='blue')
    current_r2_train = r2_score(train_labels, predict_train)
    plt.text(min(predict_train), max(train_labels), f'R²={current_r2_train:.4f}', color='blue')
    plt.title('Train Prediction vs Actual')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(fig_name_train, format='png', dpi=300)
    plt.close()
    
    
    plt.figure()
    sns.regplot(x=predict_train, y=train_labels, color='blue', label='Train')
    sns.regplot(x=predict_test, y=test_labels, color='red', label='Test')
    plt.legend()
    plt.text(min(np.concatenate([predict_train, predict_test])), max(np.concatenate([train_labels, test_labels])), f'R²={r2_score(np.concatenate([train_labels, test_labels]), np.concatenate([predict_train, predict_test])):.4f}', color='green')
    plt.title('All Prediction vs Actual')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(fig_name_all, format='png', dpi=300)
    plt.close()

    
    df_prediction_comparison_train = pd.DataFrame({
        'Predicted Train': predict_train.squeeze(),
        'Actual Train': train_labels.squeeze()
    })
    df_prediction_comparison_test = pd.DataFrame({
        'Predicted Test': predict_test.squeeze(),
        'Actual Test': test_labels.squeeze()
    })
    df_prediction_comparison_all = pd.DataFrame({
        'Predicted': np.concatenate([predict_train, predict_test]),
        'Actual': np.concatenate([train_labels, test_labels]),
        'Dataset': ['Train'] * len(train_labels) + ['Test'] * len(test_labels)
    })

    df_prediction_comparison_train.to_excel(os.path.join(results_dir, f'{mm}_SVR_predictions_train.xlsx'), index=False)
    df_prediction_comparison_test.to_excel(os.path.join(results_dir, f'{mm}_SVR_predictions_test.xlsx'), index=False)
    df_prediction_comparison_all.to_excel(os.path.join(results_dir, f'{mm}_SVR_predictions_all.xlsx'), index=False)

    
    results_df = pd.concat([results_df, pd.DataFrame([{
        'Iteration': mm,
        'target': tg,
        'R2_Score_test': current_r2_test,
        'R2_Score_train': current_r2_train,
        'Train Loss': train_mape,  
        'Test Loss': test_mape,    
        'Figure_Path_Test': fig_name_test,
        'Figure_Path_Train': fig_name_train,
        'Figure_Path_All': fig_name_all,
        'Loss_Path': excel_path
    }])], ignore_index=True)


results_summary_filename = os.path.join(results_dir, 'results_summary_SVR.csv')
results_df.to_csv(results_summary_filename, index=False)


           Ba        Ca   Sr     Ti     Zr        Sn    Hf         W  \
0    1.000000  0.000000  0.0  1.000  0.000  0.000000  0.00  1.000000   
1    1.000000  0.000000  0.0  0.950  0.050  0.000000  0.00  0.978718   
2    1.000000  0.000000  0.0  0.950  0.000  0.071429  0.00  0.965424   
3    1.000000  0.000000  0.0  0.925  0.075  0.000000  0.00  0.968220   
4    1.000000  0.000000  0.0  0.900  0.000  0.000000  0.10  0.877303   
..        ...       ...  ...    ...    ...       ...   ...       ...   
149  0.333333  0.666667  0.0  1.000  0.000  0.000000  0.00  0.664214   
150  0.333333  0.666667  0.0  0.500  0.500  0.000000  0.00  0.495194   
151  0.333333  0.666667  0.0  0.500  0.000  0.000000  0.50  0.227934   
152  0.200000  0.800000  0.0  0.840  0.000  0.000000  0.16  0.439015   
153  0.000000  1.000000  0.0  1.000  0.000  0.000000  0.00  0.496321   

           EI        EA         μ  
0    0.885689  0.786531  0.000000  
1    0.892040  0.765946  0.016596  
2    0.868796  0.725267  0.

/tmp/ipykernel_229968/2812312343.py:159: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame([{
